In [1]:
import numpy as np
import pandas as pd
pd.set_option('max_columns',90)
pd.set_option('max_rows',None)
from sklearn.preprocessing import Imputer
from sklearn.ensemble import RandomForestClassifier
from sklearn import grid_search
from sklearn import cross_validation
from sklearn import metrics
import sklearn as sk

In [2]:
filename = 'Medicare_Physician_and_Other_Supplier_NPI_Aggregate_CY2014.txt'
dataRaw = pd.read_csv(filename,sep='\t')
data = dataRaw[dataRaw.nppes_entity_code=='I']
data['pcp'] = np.logical_or(data.provider_type=='Internal Medicine',data.provider_type=='Family Practice').astype(int)

C:\Users\rek\Desktop\sw\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\rek\Desktop\sw\Anaconda\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [3]:
# data.groupby('nppes_provider_state')['pcp'].mean()
# Not much difference from one state to the next in PrimaryCareProvider percentage. So don't try to use geography as a predictor.

In [4]:
data['nppes_provider_gender'] = (data['nppes_provider_gender']=='F').astype(int)
data['medicare_participation_indicator'] = (data['medicare_participation_indicator']=='Y').astype(int)

C:\Users\rek\Desktop\sw\Anaconda\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\rek\Desktop\sw\Anaconda\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [5]:
data.drop(['provider_type','nppes_entity_code','nppes_provider_last_org_name','nppes_provider_first_name',
           'nppes_provider_mi','nppes_credentials','nppes_provider_street1','nppes_provider_street2',
           'nppes_provider_city','nppes_provider_state','nppes_provider_zip','nppes_provider_country',
           'drug_suppress_indicator','med_suppress_indicator'],
           axis=1,inplace=True)

C:\Users\rek\Desktop\sw\Anaconda\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [6]:
data.head()

,npi,nppes_provider_gender,medicare_participation_indicator,number_of_hcpcs,total_services,total_unique_benes,total_submitted_chrg_amt,total_medicare_allowed_amt,total_medicare_payment_amt,total_medicare_stnd_amt,number_of_drug_hcpcs,total_drug_services,total_drug_unique_benes,total_drug_submitted_chrg_amt,total_drug_medicare_allowed_amt,total_drug_medicare_payment_amt,total_drug_medicare_stnd_amt,number_of_med_hcpcs,total_med_services,total_med_unique_benes,total_med_submitted_chrg_amt,total_med_medicare_allowed_amt,total_med_medicare_payment_amt,total_med_medicare_stnd_amt,beneficiary_average_age,beneficiary_age_less_65_count,beneficiary_age_65_74_count,beneficiary_age_75_84_count,beneficiary_age_greater_84_count,beneficiary_female_count,beneficiary_male_count,beneficiary_race_white_count,beneficiary_race_black_count,beneficiary_race_api_count,beneficiary_race_hispanic_count,beneficiary_race_natind_count,beneficiary_race_other_count,beneficiary_nondual_count,beneficiary_dual_count,beneficiary_cc_afib_percent,beneficiary_cc_alzrdsd_percent,beneficiary_cc_asthma_percent,beneficiary_cc_cancer_percent,beneficiary_cc_chf_percent,beneficiary_cc_ckd_percent,beneficiary_cc_copd_percent,beneficiary_cc_depr_percent,beneficiary_cc_diab_percent,beneficiary_cc_hyperl_percent,beneficiary_cc_hypert_percent,beneficiary_cc_ihd_percent,beneficiary_cc_ost_percent,beneficiary_cc_raoa_percent,beneficiary_cc_schiot_percent,beneficiary_cc_strk_percent,Beneficiary_Average_Risk_Score,pcp
0,1003000126,0,1,16,2749.0,913,750652.0,254849.52,194073.09,193275.26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,2749.0,913.0,750652.0,254849.52,194073.09,193275.26,74,162.0,277.0,297.0,177.0,504.0,409.0,881.0,NaN,NaN,NaN,NaN,NaN,588.0,325.0,27.0,31.0,18.0,16.0,49.0,56.0,43.0,46.0,52.0,70.0,75.0,68.0,14.0,53.0,18.0,20.0,2.2206,1
1,1003000134,0,1,13,8643.0,4276,1357139.0,321641.88,245364.34,227205.32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,8643.0,4276.0,1357139.0,321641.88,245364.34,227205.32,76,102.0,1856.0,1500.0,818.0,2252.0,2024.0,4057.0,36.0,47.0,69.0,0.0,67.0,4132.0,144.0,12.0,7.0,4.0,13.0,11.0,14.0,7.0,12.0,20.0,53.0,57.0,28.0,9.0,37.0,1.0,3.0,1.0007,0
2,1003000142,0,1,40,654.0,165,122130.0,69771.61,51820.04,51233.02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0,654.0,165.0,122130.0,69771.61,51820.04,51233.02,60,104.0,37.0,NaN,NaN,94.0,71.0,115.0,NaN,NaN,NaN,NaN,NaN,72.0,93.0,NaN,NaN,21.0,8.0,22.0,28.0,35.0,44.0,48.0,61.0,75.0,41.0,8.0,75.0,9.0,NaN,1.7777,0
3,1003000407,0,1,39,1012.0,392,169486.0,110286.91,85665.04,87328.49,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39.0,1012.0,392.0,169486.0,110286.91,85665.04,87328.49,74,88.0,82.0,103.0,119.0,230.0,162.0,NaN,NaN,NaN,NaN,NaN,NaN,234.0,158.0,22.0,30.0,15.0,13.0,47.0,43.0,38.0,47.0,40.0,59.0,75.0,62.0,11.0,45.0,19.0,10.0,1.7228,1
4,1003000423,1,1,29,293.0,57,24148.0,12261.24,9491.61,9918.62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60,27.0,18.0,NaN,NaN,57.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,31.0,26.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.0,23.0,51.0,46.0,19.0,NaN,32.0,NaN,NaN,0.9101,0


In [7]:
# Use the same transformation on the data set as from Question 2, to remove the volume correlations.

# re-define the demographic columns to be percentages, rather than raw counts:
cols_pctof_TotUniBen = ['beneficiary_age_less_65_count','beneficiary_age_65_74_count',
                        'beneficiary_age_75_84_count','beneficiary_age_greater_84_count',
                        'beneficiary_female_count','beneficiary_male_count','beneficiary_race_white_count',
                        'beneficiary_race_black_count','beneficiary_race_api_count','beneficiary_race_hispanic_count',
                        'beneficiary_race_natind_count','beneficiary_race_other_count',
                        'beneficiary_nondual_count','beneficiary_dual_count']
colNameReplacements_dict = {x:x.replace('_count','_pct') for x in cols_pctof_TotUniBen}
for cc in cols_pctof_TotUniBen:
    data[cc] = 100.0*data[cc]/data.total_unique_benes
data.rename(columns=colNameReplacements_dict,inplace=True)

# also redefine the Medicare payment amounts as fractions rather than raw dollars:
data.total_medicare_stnd_amt = data.total_medicare_stnd_amt / data.total_medicare_payment_amt
data.total_drug_medicare_stnd_amt = data.total_drug_medicare_stnd_amt / data.total_drug_medicare_payment_amt
data.total_med_medicare_stnd_amt = data.total_med_medicare_stnd_amt / data.total_med_medicare_payment_amt
data.total_medicare_payment_amt = data.total_medicare_payment_amt / data.total_medicare_allowed_amt
data.total_drug_medicare_payment_amt = data.total_drug_medicare_payment_amt / data.total_drug_medicare_allowed_amt
data.total_med_medicare_payment_amt = data.total_med_medicare_payment_amt / data.total_med_medicare_allowed_amt
data.total_medicare_allowed_amt = data.total_medicare_allowed_amt / data.total_submitted_chrg_amt
data.total_drug_medicare_allowed_amt = data.total_drug_medicare_allowed_amt / data.total_drug_submitted_chrg_amt
data.total_med_medicare_allowed_amt = data.total_med_medicare_allowed_amt / data.total_med_submitted_chrg_amt
# normalize total services and total submitted charges as the amount per beneficiary:
data.total_submitted_chrg_amt = data.total_submitted_chrg_amt/data.total_unique_benes
data.total_drug_submitted_chrg_amt = data.total_drug_submitted_chrg_amt/data.total_drug_unique_benes
data.total_med_submitted_chrg_amt = data.total_med_submitted_chrg_amt/data.total_med_unique_benes
data.total_services = data.total_services/data.total_unique_benes
data.total_drug_services = data.total_drug_services/data.total_drug_unique_benes
data.total_med_services = data.total_med_services/data.total_med_unique_benes
colNameReplacements_dict2 = {x:x.replace('_amt','_frac') for x in data.columns}
data.rename(columns=colNameReplacements_dict2,inplace=True)
colNameReplacements_dict3 = {'total_submitted_chrg_frac':'total_submitted_chrg_amt_per_bene',
                             'total_drug_submitted_chrg_frac':'total_drug_submitted_chrg_amt_per_drug_bene',
                             'total_med_submitted_chrg_frac':'total_med_submitted_chrg_amt_per_med_bene'}
data.rename(columns=colNameReplacements_dict3,inplace=True)
colNameReplacements_dict4 = {'total_services':'total_services_per_bene',
                             'total_drug_services':'total_drug_services_per_drug_bene',
                             'total_med_services':'total_med_services_per_med_bene'}
data.rename(columns=colNameReplacements_dict4,inplace=True)

C:\Users\rek\Desktop\sw\Anaconda\lib\site-packages\ipykernel\__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\rek\Desktop\sw\Anaconda\lib\site-packages\pandas\core\frame.py:2754: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)
C:\Users\rek\Desktop\sw\Anaconda\lib\site-packages\pandas\core\generic.py:2701: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-co

In [8]:
for cc in data:
    print('%s    %s' % (np.count_nonzero(data[cc].isnull()),cc))

0    npi
0    nppes_provider_gender
0    medicare_participation_indicator
0    number_of_hcpcs
0    total_services_per_bene
0    total_unique_benes
0    total_submitted_chrg_amt_per_bene
0    total_medicare_allowed_frac
0    total_medicare_payment_frac
1    total_medicare_stnd_frac
102375    number_of_drug_hcpcs
714300    total_drug_services_per_drug_bene
102375    total_drug_unique_benes
714300    total_drug_submitted_chrg_amt_per_drug_bene
714300    total_drug_medicare_allowed_frac
714300    total_drug_medicare_payment_frac
714301    total_drug_medicare_stnd_frac
102375    number_of_med_hcpcs
102377    total_med_services_per_med_bene
102375    total_med_unique_benes
102377    total_med_submitted_chrg_amt_per_med_bene
102377    total_med_medicare_allowed_frac
102377    total_med_medicare_payment_frac
102378    total_med_medicare_stnd_frac
0    beneficiary_average_age
290742    beneficiary_age_less_65_pct
149581    beneficiary_age_65_74_pct
261826    beneficiary_age_75_84_pct
355685   

In [9]:
# Lots of nulls in this data set. 
# Mean imputation probably offers the best trade-off between efficiency and eventual classifier performance.

colsData = data.columns
imp = Imputer(missing_values=np.nan, strategy='mean', axis=0)
data = pd.DataFrame(imp.fit_transform(data),
                      columns=[colsData[x] for x in range(len(colsData)) if ~np.isnan(imp.statistics_[x])])

In [10]:
colX = [cc for cc in data.columns if cc != 'pcp']
X = data[colX]
y = data['pcp']

In [11]:
indices = np.random.permutation(xrange(len(y)))
X,y = X.iloc[indices], y.iloc[indices]

In [12]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.3, random_state=21)
X.shape, X_train.shape, X_test.shape

((925347, 54), (647742, 54), (277605, 54))

In [13]:
cx = RandomForestClassifier(n_estimators=20)
gs = grid_search.GridSearchCV(
    cx,
    {'min_samples_leaf': [20,50,100,200,500],
     'max_depth': [10,20,50]},
    cv=5,  # 5-fold cross validation
    n_jobs=3,  # run each hyperparameter in one of two parallel jobs
    scoring='roc_auc'
)

In [14]:
%time gs.fit(X_train, y_train)

Wall time: 28min 39s


GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params={}, iid=True, n_jobs=3,
       param_grid={'max_depth': [10, 20, 50], 'min_samples_leaf': [20, 50, 100, 200, 500]},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [15]:
print gs.best_params_

{'max_depth': 50, 'min_samples_leaf': 20}


In [ ]:
# The 5-fold cross-validation grid search chose the highest available max_depth and smallest min_samples_leaf.
# Therefore the model may be biased and underfitting the data.
    # With more time, I would try smaller values for min_samples_leaf and larger values for max_depth.
        # These might improve results further.
        # Still, the area under the ROC curve computed below is 0.94897
            # This is better than I had been expecting for such a noisy data set.

In [20]:
testPP = gs.predict_proba(X_test)

In [23]:
metrics.roc_auc_score(y_test, [jj[1] for jj in testPP])

0.94897424537504516